For writing the narrative and exporting data.

In [8]:
import pickle
import math
import json
import numpy as np
import matplotlib.pyplot as plt

CUNAME = 'Columbia University in the City of New York'
S_TOTALS = pickle.load(open('../data/S_Sex_Graduates.p', 'rb'))
E_TOTALS = pickle.load(open('../data/E_Sex_Graduates.p', 'rb'))
D_TOTALS = pickle.load(open('../data/SE-Detailed_Sex.p', 'rb'))

years = sorted(S_TOTALS.keys())
institutions = sorted(S_TOTALS[ max(years) ].keys())
fields = {
    'Science': set(),
    'Engineering': set()
}
for year in years:
    inst_stats = D_TOTALS[year]['Columbia University in the City of New York']
    for seh in inst_stats.keys():
        for field in inst_stats[seh].keys():
            fields[seh].add(field)
fields['Science'] = sorted(fields['Science'])
fields['Engineering'] = sorted(fields['Engineering'])

print(f'{len(institutions)} institutions, {len(fields["Science"]) + len(fields["Engineering"])} fields.')

11 institutions, 39 fields.


In [13]:
# Returns female and male headcounts for a field in an institution
def get_counts(seh, field, inst='Columbia University in the City of New York', default=None):
    females = []
    males = []
    for year in years:
        stats = D_TOTALS[year][inst][seh].get(field, None)
        if stats is None:
            females.append(default)
            males.append(default)
            continue
        females.append(stats.get('Female', 0))
        males.append(stats.get('Male', 0))
    return females, males

complete_fields = {
    'Science': [],
    'Engineering': [],
}
topic_lookup = {}
for topic in complete_fields.keys():
    for field in fields[topic]:
        females, males = get_counts(topic, field)
        if None not in females and None not in males:
            complete_fields[topic].append(field)
        topic_lookup[field] = topic
print(f'{len(complete_fields["Science"]) + len(complete_fields["Engineering"])} complete fields.')

19 complete fields.


In [15]:
# Convenient function for making subplots
def subplots(num=1, cols=3, figsize=6):
    if num < cols:
        cols = num
    rows = math.ceil(num / cols)
    return plt.subplots(rows, cols, figsize=(figsize * cols, figsize * rows))

# Gets female and male headcounts for a field in an institution
def get_counts(field, inst='Columbia University in the City of New York'):
    females = []
    males = []
    topic = topic_lookup[field]
    for year in years:
        stats = D_TOTALS[year][inst][topic].get(field, None)
        if stats is None:
            females.append(default)
            males.append(default)
            continue
        females.append(stats.get('Female', 0))
        males.append(stats.get('Male', 0))
    return females, males

# Gets proportions for a field
def get_proportions(field, inst='Columbmia University in the City of New York'):
    females, males = get_counts(field, inst)
    proportions = []
    for i in range(len(years)):
        f, m = females[i], males[i]
        if f is None or m is None: 
            proportions.append(None)
        else:
            proportions.append(f / (m+f))
    return proportions

# Gets total populations for a field
def get_total_counts(field, inst='Columbia University in the City of New York'):
    females, males = get_counts(field, inst)
    return [f + m for f, m in zip(females, males)]

# Plots proportions for a field
def plot_proportions(ax, field, inst='Columbia University in the City of New York', title=None, peers=False,
                     start=1994, label_line=True):
    proportions = get_proportions(field, inst)
    if peers:
        for inst in institutions:
            if inst != CUNAME:
                ax.plot(years, get_proportions(field, inst), color='lightgreen', linewidth=1)
    start_i = start - years[0]
    # ax.plot(years[:start_i + 1], proportions[:start_i + 1], color='lightgreen')
    ax.plot(years[start_i:], proportions[start_i:], color='green')
    if label_line: ax.annotate(field, (years[start_i], proportions[start_i]))
    ax.set_ylim(bottom=0, top=1)
    ax.set_title(title or field)

# Plots female and male headcounts for a field
def plot_counts(ax, field, inst='Columbia University in the City of New York', title=None):
    females, males = get_counts(field, inst)
    ax.plot(years, females, label='Female')
    ax.plot(years, males, label='Male')
    ax.legend()
    ax.set_ylim(bottom=0)
    ax.set_title(title or field)
    ax.set_ylabel('Number of people')

# Plots triplet for a field
def plot_pair(ax_ary, index, field):    
    if index == None:
        axs = [ax_ary[i] for i in range(2)]
    else:
        axs = [ax_ary[index, i] for i in range(2)]
    plot_proportions(axs[0], field, peers=True)
    plot_counts(axs[1], field)

# Plots triplet for multiple fields
def plot_selected_pairs(selected_fields):
    num_fields = len(selected_fields)
    _, ax = subplots(num_fields*2, cols=2)
    if num_fields == 1:
        plot_pairs(ax, None, selected_fields[0])
        return
    for i, field in enumerate(selected_fields):
        plot_pairs(ax, i, field)

# Gets overall counts from complete fields for an institution in a topic
def get_overall_counts(topic, inst='Columbia University in the City of New York'):
    overallF = [0 for _ in range(len(years))]
    overallM = [0 for _ in range(len(years))]
    counts = {field: get_counts(field, inst, default=0) for field in complete_fields[topic]}

    for field in complete_fields[topic]:
        f, m = counts[field]
        for i in range(len(years)):
            overallF[i] += f[i]
            overallM[i] += m[i]
    return overallF, overallM

# Gets overall proportions from overall counts
def get_overall_proportions(topic, inst='Columbia University in the City of New York'):
    overallF, overallM = get_overall_counts(topic, inst)
    return [0 if m+f == 0 else f/(m+f) for f, m in zip(overallF, overallM)]

# Plots proportions for a topic
def plot_topic_proportions(ax, topic, peers=False, title=None, label=[]):
    colors = ['blue', 'lightblue']
    if topic == 'Science':
        colors = ['red', 'pink']
    if peers:
        for inst in institutions:
            if inst != CUNAME:
                ax.plot(years, get_overall_proportions(topic, inst), color=colors[1], linewidth=1)
    cu_props = get_overall_proportions(topic, CUNAME)
    ax.plot(years, cu_props, color=colors[0], linewidth=2)
    for i in [0, -1] + [year - years[0] for year in label]:
        ax.annotate("{0:.0%}".format(cu_props[i]), (years[i], cu_props[i]))
    ax.set_ylim(bottom=0, top=1)
    ax.set_title(title or topic)

# Plots a female and male headcounts for a topic
def plot_topic_counts(ax, topic, inst='Columbia University in the City of New York', title=None):
    females, males = get_overall_counts(topic, inst)
    ax.plot(years, females, label='Female')
    ax.plot(years, males, label='Male')
    ax.legend()
    ax.set_ylim(bottom=0)
    ax.set_title(title or topic)
    ax.set_ylabel('Number of people')

# Science

Constant: anthropology, biology, chemistry, sociology, **statistics**
    
Converging: **math and applied math**

Diverging: **computer science**, **physics**


### Growing fields

Converging: BME, 

Diverging (female): Biometry and epidemiology, social sciences not elsewhere classified

Diverging (male): Biophysics, civil engineering,

Constant: ChemE, earth atmospheric and ocean sciences not elsewhere classified

### Shrinking fields

Diverging (female): biosciences not elsewhere classified, political science and government

Converging: Nursing science, percentative medicine and community health

### A problem

Engineering not elsewhere classified has data since 2014.

Neurobiology and neuroscience has data since 2012.